# Multi Feature-Rich Synthetic Colour (MFRSC) PARA NUBES DE PUNTOS

Este es un corto tutorial para explicar paso a paso la generación de color sintético para nube de puntos a partir de diversas características acorde el trabajo:

- Balado, J., González, E., Rodríguez-Somoza, J. L., & Arias, P. (2023). Multi feature-rich synthetic colour to improve human visual perception of point clouds. ISPRS Journal of Photogrammetry and Remote Sensing, 196, 514-527.

# Importación de librerías

Las librerías empleadas en la morfología matemática son *numpy*, *pyntcloud* y *pandas*.

- https://numpy.org/
- https://pyntcloud.readthedocs.io/en/latest/#
- https://pandas.pydata.org/


In [ ]:
import numpy as np
from pyntcloud import PyntCloud
import pandas as pd

# Lectura de datos

Aunque existen numerosos formatos de nubes de puntos, en este tutorial vamos a partir de nubes en formato *txt*, que pueden ser leídas mediante *numpy*, puesto que no tienen ningún tipo de compresión. La nube en *txt* se estructura en 1 punto por fila y un atributo por columna, además se especifica ' ' como delimitador entre columnas. 

En este caso, la nube de entrada empleada para colorear contiene por este orden: 
- Coordenadas XYZ
- Intensidad
- Número de retorno  

Como ejemplo de trabajo se proporciona una nube escaneada con Mobile Laser Scanner.

In [ ]:
# Lectura de datos
input_data = np.loadtxt("Nubes/pointcloud_XYZ_I_Rn.txt", delimiter=' ')

# Extracción y conversión de datos

Dada que el coloreado se basa en características, es necesario previamente extraer algunas de ellas. La librería*pyntcloud* ofrece unas funcionas fáciles y directas para la extracción de características geométricas, pero es necesaria previamente la conversión de los datos de entrada a objeto-nube de puntos.

Para los no familiarizados con el acceso a los datos de nubes de puntos, tanto en *pandas* como en otras librerías, el acceso se realiza mediante [n filas, n columnas], de tal forma que con ":" se indica que se seleccionan todas las filas o columnas, y numéricamente mediante "n:m" se indica que se accede desde la fila/columna "n+1" a la m.

In [ ]:
# Extraer matriz de coordenadas de los datos de entrada
coord = pd.DataFrame(list(zip(input_data[:,0],input_data[:,1],input_data[:,2])))  

# Asignar título a columnas
coord.columns =['x', 'y', 'z']

# Visualización
print(coord)

Convertir coordenadas a objeto nube de puntos

In [ ]:
# Conversión
cloud = PyntCloud(coord)

# Visualización
print(cloud)

# Extracción de características a partir de la nube de entrada

Algunas de las características que emplean el MFRSC están disponibles directamente a partir de los datos de entrada:

- ***Intensidad de reflectancia*** (Re) es una característica radiométrica proporcionada por el sensor LiDAR. La intensidad se utiliza ampliamente para identificar superficies con alta reflectividad, como marcas viales y señales de tráfico, así como para preservar texturas debido a variaciones en el material y la rugosidad.

- ***Número de retorno*** (Rn) es una característica radiométrica relacionada con la capacidad de penetración del láser en elementos vegetales y cristales en función de su longitud de onda. La característica del número de retorno sólo está disponible en LiDAR multirretorno y el número máximo de retornos suele ser cinco en los nuevos sistemas LiDAR. Esta característica se utiliza ampliamente en la identificación de la cubierta vegetal.

- ***Profundidad*** (De) es la característica que proporciona una visualización de las distancias horizontales. La profundidad es una característica muy útil para identificar objetos por su silueta (diferencia de distancia entre el objetivo y el fondo).

- ***Altura*** (He) es una característica proporcionada por la nube de puntos en coordenada Z. La altura puede medirse desde el nivel del mar, en caso de datos georreferenciados. En cualquiera de estas situaciones, en la fase de normalización posterior se elimina el desfase del nivel del mar. La visualización de la altura como un gradiente de color permite identificar la verticalidad y horizontalidad de los elementos del entorno, un principio de la psicología humana para interpretar las escenas.

In [ ]:
# Reflectancia
Intens = input_data[:,3]

# Numero retornos
Return = input_data[:,4]

# Profundidad
Depth = np.sqrt(input_data[:,0]**2 + input_data[:,1]**2);

# Altura
Height = input_data[:,2];

# Extracción de características a partir de distancias entre puntos

Para el siguiente conjunto de características es necesario calcular las vecindades entre puntos. La vecindad se calcula para *k* = 25 vecinos para obtener características geométricas en los pasos siguientes, aunque para la densidad solo hacen falta los 5 vecinos más próximos.

In [ ]:
# Cálculo de 25 vecinos
k_neighbors_25 = cloud.get_neighbors(k=25)

- ***Densidad de puntos*** (Pd) es una característica que depende de la frecuencia de barrido láser, la distancia entre el láser y la superficie objetivo y el ángulo de incidencia. Es una característica relevante para resaltar o atenuar puntos aislados o áreas con baja densidad de puntos. En este trabajo calculamos la densidad de puntos basándonos en la distancia entre el primer (d1) y el cuarto (d4) puntos vecinos más cercanos. Esta medida fue propuesta por Pfeifer et al. (2021). Sin embargo, para obtener los valores delimitados (0-1), se ha invertido la división.

In [ ]:
# Distancia al primer vecino
d1 = np.sqrt(np.sum((input_data[:,0:3]-input_data[k_neighbors_25[:,0],0:3])**2,axis=1))

# Distancia al quinto vecino
d5 = np.sqrt(np.sum((input_data[:,0:3]-input_data[k_neighbors_25[:,4],0:3])**2,axis=1))

# Cálculo de densidad
Density = d1/d5

# Extracción de características a partir de normales

Conocer la orientación de la superficies que conforman los puntos es fundamental para estimar la inclinación: 

- ***Inclinación*** (In) es la característica que indica la orientación de la superficie que contiene el punto con respecto al horizonte. La inclinación se obtiene a partir del cálculo de la normal de superficie del punto respecto a *k* vecinos más próximos. El primer modelo de reconocimiento basado en rasgos, conocido como el pandemonio propuesto por Selfridge (1988), prevé que el sistema visual puede disponer de detectores de rasgos geométricos simples, como líneas verticales, horizontales y oblicuas. Por lo tanto, la visualización de la inclinación de las superficies en la nube de puntos puede ayudar a un reconocimiento más directo.

In [ ]:
# Cálculo de normales
cloud.add_scalar_field("normals", k_neighbors=k_neighbors_25)

# Extracción de las normales del objeto nube de puntos
normals = cloud.points[["nx(26)", "ny(26)", "nz(26)"]].to_numpy()

# Cálculo de la inclinación
Inclination = np.absolute(np.arctan(np.sqrt(normals[:,0]**2+normals[:,1]**2)/normals[:,2]))*180/np.pi

# Extracción de características a partir de autovalores

Las características restantes se obtienen a partir de los autovalores de la nube de puntos:


- ***Linealidad*** (Li) es una característica geométrica basada en la distribución de *k* puntos vecinos a partir de los valores propios (Weinmann et al., 2015). La linealidad realza elementos lineales, por ejemplo, objetos similares a postes, pero también esquinas entre dos planos. Según muchas teorías de reconocimiento de objetos, los bordes son una de las características más importantes. Según la teoría de Marr y Nishihara, en la primera etapa de la percepción, la imagen se describe como bordes, manchas, barras y la distribución geométrica. Según la teoría del reconocimiento por componentes (RBC) de Biederman, los geones son formas volumétricas simples y son los responsables del reconocimiento de objetos. El primer paso del reconocimiento consistiría en extraer los bordes a partir de los cambios de luminancia y, paralelamente, la división del objeto en regiones cóncavas. El modelo RBC/JIM (Kurbat, 1994; Hummel y Biederman, 1992) considera que el reconocimiento se produce de forma similar a las redes neuronales: activación de neuronas en capas sucesivas. En la primera capa, se detectan los bordes. En las capas 2 y 3, geones, simetría y manchas. En las capas 4 y 5, tamaño y orientación de los geones. Dada la importancia de los bordes, la linealidad es una característica muy relevante para la identificación de objetos.

- ***Planaridad*** (Pl) es una característica geométrica calculada a partir de los valores propios (Weinmann et al., 2015). La planaridad realza los elementos planos que conforman la mayor parte del entorno construido y permiten visualizar la curvatura. Según la teoría RBC, además de la detección de bordes, en la construcción de representaciones 3D también son relevantes las propiedades no accidentales (simetría, paralelismo, rectitud/curvatura y conexiones), responsables de mantener la constancia de los objetos. Por lo tanto, la planaridad es relevante para la identificación de geones debido a la visualización de la curvatura de los objetos.

- ***Dispersión*** (Sc) es una característica geométrica calculada a partir de los valores propios (Weinmann et al., 2015). La dispersión realza los elementos de formas 3D irregulares. Esta categoría incluye la vegetación, así como las uniones de tres o más planos, de modo que la dispersión se presenta como una característica también alineada con la identificación de bordes y geones.

In [ ]:
# Cálulo de autovalores
eigenvalues = cloud.add_scalar_field("eigen_values", k_neighbors=k_neighbors_25)

# Cálculo de características
cloud.add_scalar_field("linearity", ev=eigenvalues)
cloud.add_scalar_field("planarity", ev=eigenvalues)
cloud.add_scalar_field("sphericity", ev=eigenvalues)

# Visualización de los datos
cloud.points

Como se desprende de la justificación anterior, no existe una correspondencia unívoca entre las características y los descriptores perceptivos. Aunque las características de las nubes de puntos se eligen cuidadosamente, muchas de ellas tienen interdependencias entre sí y abarcan varios descriptores perceptuales:

|   | Edges | Texture | Shape | Size | Depth | Orientation |
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- |
| Reflectance |   | x | | | | x |
| Return number |   | x | | | |  |
| Depth | x |  | | x |x | x |
| Height | x |  | | x | x | x |
| Point density | x  | x | | | | |
| Inclination |   |  | x | | | x |
| Linearity |  x |  | x | | | |
| Planarity |  x |  | x | | | |
| Scattering |  x |  | x | | | |

# Normalización de características

Las características seleccionadas tienen diferentes rangos, por lo que es necesario un proceso de normalización para acotar los valores entre 0 y 1. Algunas características como la densidad, la linealidad, la planaridad y la dispersión ya se encuentran en este rango. Para el resto, se aplica una función de normalización. Para la intensidad y el número de retorno, a mayores hay que definir umbrales de saturación. 

In [ ]:
# Normalizar

Intens_n = Intens/1500

Return_n = Return/4

Depth_n = (Depth-np.min(Depth))/(np.max(Depth)-np.min(Depth))

Height_n = (Height-np.min(Height))/(np.max(Height)-np.min(Height))

Inclination_n = Inclination/90

# Generación de la matriz de características

Para mayor comodidad y posibilidad de una posible exportación posterior, las características calculadas se juntan en una única matriz.


In [ ]:
# Matriz de features
Features = np.column_stack((Intens_n,Return_n,Depth_n,Height_n,Density, cloud.points[['linearity(26)','planarity(26)','sphericity(26)']].to_numpy(),Inclination_n))


# Combinación de características por canal RGB

Existen 362880 posibles combinaciones de todas las características sin repetirlas. Valorar la combinación de todas para cada caso de estudio conlleva varias horas de procesado, por lo que a partir del paper que presenta el MFRSC, se han testeado todas las posibles combinaciones y sugerido un orden óptimo de características.

La reducción se realiza mediante la conversión de 3 características a 1, según la conversión de color RGB a escala de grises.



<center> <img src="Figures/F02MFRSC.jpg"></center>
<center> Figura 1. Combinación de caracteristicas a tres canales RGB  </center>

In [ ]:
# RGB
# Depth-Linear-Height
R = 0.2989 * Features[:,2] + 0.5870 * Features[:,5] + 0.1140 * Features[:,3];

# Return-Reflectance-Inclination 
G = 0.2989 * Features[:,1] + 0.5870 * Features[:,0] + 0.1140 * Features[:,8];

# Planar-Scatter-Density
B = 0.2989 * Features[:,6] + 0.5870 * Features[:,7] + 0.1140 * Features[:,4];

# Exportar nube de puntos con MFRSC

Para guardar la nube en disco, se recurre a la librería *numpy* y a un formato de la nube en *txt*. Para el guardado se indica una dirección y nombre del archivo a generar. La nube de puntos de salida contiene la geometría (XYZ) de la nube de entrada y los colores (RGB) calculados en escala 0-1


In [ ]:
# Definición de la ruta y nombre del archivo
ruta = "Nubes/pointcloud_MFRSC.txt"

#Selección de datos 
output_data = np.column_stack((input_data[:,0:3],R,G,B))

# Guardado
np.savetxt(ruta,output_data,delimiter=' ') 

Con esto concluye el tutorial para generar MFRSC para nubes de puntos. Recomiendo activamente la descarga de la nube y su visualización en Cloud Compare. A mayores, el ultimo fragmento de código es la generación de una función para emplearla en script propios sin explicación paso a paso ni salidas intermedias:

In [ ]:
def MFRSC(XYZ,Intens,Return,Sat_Intens, Sat_Rn):
    # Extraer matriz de coordenadas de los datos de entrada
    coord = pd.DataFrame(list(zip(XYZ[:,0],XYZ[:,1],XYZ[:,2])))  

    # Asignar título a columnas
    coord.columns =['x', 'y', 'z']
    
    # Conversión
    cloud = PyntCloud(coord)

    # Profundidad
    Depth = np.sqrt(XYZ[:,0]**2 + XYZ[:,1]**2);

    # Altura
    Height = XYZ[:,2];
    
    # Cálculo de 25 vecinos
    k_neighbors_25 = cloud.get_neighbors(k=25)

    # Distances
    d1 = np.sqrt(np.sum((XYZ[:,0:3]-XYZ[k_neighbors_25[:,0],0:3])**2,axis=1))    
    d5 = np.sqrt(np.sum((XYZ[:,0:3]-XYZ[k_neighbors_25[:,4],0:3])**2,axis=1))

    # Cáclulo de densidad
    Density = d1/d5

    # Cálculo de normales
    cloud.add_scalar_field("normals", k_neighbors=k_neighbors_25)

    # Extracción de las normales del objeto nube de puntos
    normals = cloud.points[["nx(26)", "ny(26)", "nz(26)"]].to_numpy()

    # Cálculo de la inclinación
    Inclination = np.absolute(np.arctan(np.sqrt(normals[:,0]**2+normals[:,1]**2)/normals[:,2]))*180/np.pi

    # Cálulo de autovalores
    eigenvalues = cloud.add_scalar_field("eigen_values", k_neighbors=k_neighbors_25)

    # Cálculo de características
    cloud.add_scalar_field("linearity", ev=eigenvalues)
    cloud.add_scalar_field("planarity", ev=eigenvalues)
    cloud.add_scalar_field("sphericity", ev=eigenvalues)

    # Normalizar
    Intens_n = Intens/Sat_Intens
    Return_n = Return/Sat_Rn
    Depth_n = (Depth-np.min(Depth))/(np.max(Depth)-np.min(Depth))
    Height_n = (Height-np.min(Height))/(np.max(Height)-np.min(Height))
    Inclination_n = Inclination/90

    Features = np.column_stack((Intens_n,Return_n,Depth_n,Height_n,Density, cloud.points[['linearity(26)','planarity(26)','sphericity(26)']].to_numpy(),Inclination_n))

    # Depth-Linear-Height
    R = 0.2989 * Features[:,2] + 0.5870 * Features[:,5] + 0.1140 * Features[:,3];

    # Return-Reflectance-Inclination 
    G = 0.2989 * Features[:,1] + 0.5870 * Features[:,0] + 0.1140 * Features[:,8];

    # Planar-Scatter-Density
    B = 0.2989 * Features[:,6] + 0.5870 * Features[:,7] + 0.1140 * Features[:,4];

    #Seleccion de datos 
    output_data = np.column_stack((XYZ[:,0:3],R,G,B))

    return output_data

In [ ]:
# Ejemplo de aplicación

# Definir inputs
XYZ = input_data[:,0:3]
Intens = input_data[:,3]
Return = input_data[:,4]

# Llamada a MFRSC
output_data = MFRSC(XYZ,Intens,Return,1500, 4)

# Guardado
ruta = "Nubes/pointcloud_MFRSC.txt"
np.savetxt(ruta,output_data,delimiter=' ') 